In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
# q = bq.query(
# """
# --- your query here ---
# """)
# q.to_dataframe()
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()


,num_rows
0,3233


In [3]:
#Q1: what is the geo_id for Dane county? (note that Madison is in Dane county).
q1 = bq.query(
"""
select geo_id, state_fips_code 
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""", job_config=bigquery.QueryJobConfig(use_query_cache=False))
q1.to_dataframe().iloc[0]["geo_id"]

'55025'

In [4]:
#Q2: how many counties are there per state?
# contruct dict mapping names: num_counties
q2 = bq.query(
"""
select state_fips_code, count(*) as num_counties
from bigquery-public-data.geo_us_boundaries.counties
group by state_fips_code
order by num_counties DESC
LIMIT 5
""", job_config=bigquery.QueryJobConfig(use_query_cache=False))
dict_2 = q2.to_dataframe().to_dict("index")
result = {}
for key in dict_2:
    result[dict_2[key]["state_fips_code"]] = dict_2[key]["num_counties"]
result

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
#Q3: about how much should the queries for the last two questions cost?
result = {} # maps query : cost/TB
queries = [q1,q2]
keys = ["q1", "q2"]
# IOWA is 6.25/TB
for i in range(len(queries)):
    cost = 6.25 * (queries[i].total_bytes_billed / (1024**4)) # num of TB q1 uses
    result[keys[i]] = cost

result

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [6]:
ds = bigquery.Dataset("cs544fall.p8")

In [7]:
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('cs544fall', 'p8'))

In [8]:
for ds in bq.list_datasets("cs544fall"):
    print(ds.dataset_id)

p8
pandasp8


In [9]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
url = "gs://pandasp8/hdma-wi-2021.parquet"
job = bq.load_table_from_uri(url, "cs544fall.p8.hdma", job_config=config)
job.result()

LoadJob<project=cs544fall, location=US, id=acfa8ea6-2506-4333-bd50-d9c5cc713d5d>

In [10]:
#Q4: what are the datasets in your GCP project?
[ds.dataset_id for ds in bq.list_datasets("cs544fall")]

['p8', 'pandasp8']

In [11]:
q5 = bq.query("""
    SELECT counties.county_name, count(*) as count_loans
    FROM cs544fall.p8.hdma as hdma
    INNER JOIN bigquery-public-data.geo_us_boundaries.counties as counties
    ON counties.county_fips_code = hdma.county_code
    GROUP BY counties.county_name
    ORDER BY count_loans DESC
    LIMIT 10
""").to_dataframe()
q5

,county_name,count_loans
0,Milwaukee,46570
1,Dane,38557
2,Waukesha,34159
3,Brown,15615
4,Racine,13007
5,Outagamie,11523
6,Kenosha,10744
7,Washington,10726
8,Rock,9834
9,Winnebago,9310


In [12]:
#Q5: how many loan applications are there in the HDMA data for each county?
wi_counties_count = q5.to_dict(orient = "split", index=False)["data"]
dict(wi_counties_count)

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [13]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("cs544fall.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [14]:
#Q6: how many applications are there with your chosen income?
q6 = bq.query("""
SELECT COUNT(*) as count_applications
FROM cs544fall.p8.applications as applications
WHERE income = 110000
""").to_dataframe()
q6.iloc[0]["count_applications"]

2

In [15]:
q7 = bq.query("""
SELECT counties.county_name, COUNT(*) 
FROM cs544fall.p8.applications as applications CROSS JOIN bigquery-public-data.geo_us_boundaries.counties as counties
WHERE counties.state_fips_code = '55' AND ST_Within(ST_GEOGPOINT(applications.longitude, applications.latitude), counties.county_geom)
GROUP BY counties.county_name
""")
#q7.to_dataframe()

In [16]:
#Q7: how many applications are there in the Google sheet per WI county?
dict(q7.to_dataframe().to_dict(orient = "split", index=False)["data"])

{'Dane': 70,
 'Ozaukee': 1,
 'Door': 20,
 'Jefferson': 2,
 'Monroe': 2,
 'Sheboygan': 7,
 'Columbia': 4,
 'Adams': 1,
 'St. Croix': 1,
 'Marinette': 3,
 'Oneida': 3,
 'Bayfield': 5,
 'Kewaunee': 3,
 'Barron': 2,
 'Walworth': 4,
 'Ashland': 2,
 'Manitowoc': 1,
 'Wood': 2,
 'Winnebago': 3,
 'Iowa': 1,
 'Green Lake': 2,
 'Brown': 6,
 'Milwaukee': 4,
 'Chippewa': 1,
 'Kenosha': 1,
 'Sauk': 2,
 'Fond du Lac': 1,
 'Price': 1,
 'Outagamie': 1,
 'Dodge': 1,
 'Marathon': 1,
 'Polk': 1,
 'Douglas': 1,
 'Eau Claire': 1,
 'Crawford': 1}

In [17]:
import time
condition = True
while condition:
    # if <your condition here>:  # Hint: use bq.list_models()
    for model in bq.list_models("p8"): # if empty for loop body wont execute(?)
        condition = False
    time.sleep(5)

In [18]:
%load_ext google.cloud.bigquery

In [19]:
%%bigquery
CREATE OR REPLACE MODEL `p8.lr`
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["loan_amount"])
AS
SELECT income, loan_term, loan_amount
FROM `p8.hdma`

Query is running:   0%|          |

""


In [20]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL `p8.lr`, (
    SELECT income, loan_term, loan_amount
    FROM `p8.hdma`
))

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
#Q8: what is your model's r2_score on the HDMA dataset on which it was trained?
df.at[0, "r2_score"]

0.29165412288019577

In [22]:
%%bigquery df9
SELECT *
FROM ML.WEIGHTS(MODEL `p8.lr`)

Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
#Q9: what is the coefficient weight on the income column?
df9.at[0, "weight"]

0.8057773037176394

In [24]:
%%bigquery df10
SELECT *
FROM ML.PREDICT(MODEL `p8.lr`, (
    select loan_amount, income, 360 as loan_term
    from p8.applications
))

Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
#Q10: what ratio of the loan applications in the Google form are for amounts greater than the model would predict, given income?
# (loan_amount > predicted_loan_amount)/num_loans
count = 0
for index, row in df10.iterrows():
    if row["predicted_loan_amount"] > row["loan_amount"]:
        count += 1

count / len(df10)

0.4